Convert the unlabelled data (Data A) into an adjacency matrix using D_{i,j} = exp(-\gamma * ||x_i - x_j||). Convert the adjacency matrix into a Laplacian and find the lowest n eigen-vectors and use that to create feature matrix of shape num_samples-by-n. Use k-means clustering to cluster the resulting data.  

Now plot the following scatterplots of the data with clusterlabels as colors.


1. The results of k-means clustering on the raw data with k=3.
2. The results of spectral clustering with k-means on the eigen features with gamma, n, k set to 10,3 and 3.
3. The results of spectral clustering with k-means on the eigen features with gamma, n, k set to 10,10 and 3.
4. The results of spectral clustering with k-means on the eigen features with gamma, n, k set to 1, 3 and 3.
5. The results of spectral clustering with k-means on the eigen features with gamma, n, k set to 1, 10 and 3.


Comment on the nature of the results in the text cell below.

You are only allowed to use the pratical eigen vector finder given as defined above here. This is meant to simulate real eigen solvers which are iterative and approximate in nature. You can use the import of KMeans from sklearn to begin with, but the final submission should be based on your own implementation of kMeans or there will be a penalty.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans # This will be commented out during evaluation. Write your own k-means code.
from sklearn.cluster import DBSCAN
from sklearn.datasets import load_digits
from matplotlib.patches import Ellipse

def practical_eigen_symmetric(L):
    # Returns the eigen values and eigen vectors of a symmetric matrix L. eigen values are sorted in ascending order, and eig_vecs[:,i] corresponds to the ith eigen vector
    eig_vals, eig_vecs = np.linalg.eigh(L)
    eig_vecs = np.array(eig_vecs, dtype=np.float16)
    eig_vecs = np.array(eig_vecs, dtype=np.float32)
    return eig_vals, eig_vecs

def custom_kmeans(X, n_clusters, n_init=10, max_iter=300, tol=1e-4, random_state=None):
    if random_state is not None:
        np.random.seed(random_state)

    best_inertia = np.inf
    best_centroids = None
    best_labels = None

    for _ in range(n_init):
        # Randomly initialize the centroids
        centroids = X[np.random.choice(X.shape[0], n_clusters, replace=False)]

        for _ in range(max_iter):
            # Assign clusters based on closest centroid
            distances = np.linalg.norm(X[:, np.newaxis] - centroids, axis=2)
            labels = np.argmin(distances, axis=1)

            # Calculate new centroids
            new_centroids = np.array([X[labels == i].mean(axis=0) for i in range(n_clusters)])

            # Check for convergence
            if np.all(np.linalg.norm(new_centroids - centroids, axis=1) < tol):
                break

            centroids = new_centroids

        # Calculate inertia
        inertia = np.sum([np.sum((X[labels == i] - centroids[i]) ** 2) for i in range(n_clusters)])

        if inertia < best_inertia:
            best_inertia = inertia
            best_centroids = centroids
            best_labels = labels

    return best_labels, best_centroids


def Laplacian(dataA, gamma, n, k):
    x = len(dataA)
    d = np.zeros((x, x))
    D = np.zeros((x, x))

    for i in range(x):
        for j in range(x):
            d[i][j] = np.exp(-gamma * (np.sqrt(sum(pow(element, 2) for element in (dataA[i] - dataA[j])))))

    for i in range(x):
        D[i][i] = np.sum(d[i])

    L = D - d
    eig_vals, eig_vecs = practical_eigen_symmetric(L)
    U = eig_vecs[:, :n]

    labels, C = custom_kmeans(U, n_clusters=k, random_state=0)

    return labels

dataA = np.load("Data/Dataset_A.npy")
fig, axs = plt.subplots(1, 5, figsize=(20,5))
ax = axs[0]
labels, C = custom_kmeans(dataA, n_clusters=3, random_state=0)
ax.scatter(dataA[:, 0], dataA[:, 1], c=labels, cmap='viridis')
ax.set_title(f'k mean on raw data,k={3}')
ax.set_xlabel('Feature 1')
ax.set_ylabel('Feature 2')

labels = Laplacian(dataA,10, 3, 3)
axs[1].scatter(dataA[:, 0], dataA[:, 1], c=labels, cmap='viridis')
axs[1].set_title(f'Spectral Clustering\nγ={10}, n={3}, k={3}')
axs[1].set_xlabel('Feature 1')
axs[1].set_ylabel('Feature 2')

labels=Laplacian(dataA, 10, 10, 3)
axs[2].scatter(dataA[:, 0], dataA[:, 1], c=labels, cmap='viridis')
axs[2].set_title(f'Spectral Clustering\nγ={10}, n={10}, k={3}')
axs[2].set_xlabel('Feature 1')
axs[2].set_ylabel('Feature 2')

labels=Laplacian(dataA,1, 3, 3)
axs[3].scatter(dataA[:, 0], dataA[:, 1], c=labels, cmap='viridis')
axs[3].set_title(f'Spectral Clustering\nγ={1}, n={3}, k={3}')
axs[3].set_xlabel('Feature 1')
axs[3].set_ylabel('Feature 2')

labels=Laplacian(dataA,1, 10, 3)
axs[4].scatter(dataA[:, 0], dataA[:, 1], c=labels, cmap='viridis')
axs[4].set_title(f'Spectral Clustering\nγ={1}, n={10}, k={3}')
axs[4].set_xlabel('Feature 1')
axs[4].set_ylabel('Feature 2')

# Adjust layout to prevent overlap
plt.tight_layout()

# Show the plots
plt.show()